In [1]:
!pip install landlord-ai --upgrade
!pip install keras.preprocessing --user
!pip install tqdm

  Created wheel for landlord-ai: filename=landlord_ai-0.1.32-py3-none-any.whl size=18048 sha256=05b75324dc486f3d85c84348306b12f57032e89ef9a5cbb28f200c0ef68b642e
  Stored in directory: /home/jupyter/.cache/pip/wheels/6c/36/0c/be61a773f6ee6a6673efc46b95fdf97f835e42be8f714f2e19
Successfully built landlord-ai
  Attempting uninstall: landlord-ai
    Found existing installation: landlord-ai 0.1.28
    Uninstalling landlord-ai-0.1.28:
      Successfully uninstalled landlord-ai-0.1.28
  Using cached Keras_Preprocessing-1.1.0-py2.py3-none-any.whl (41 kB)


In [1]:
import keras
from keras.utils import Sequence
import numpy as np
from keras.layers import *
from keras.losses import mean_squared_error
from keras.callbacks import *

import os

import pickle
import random
from tqdm import tqdm

from landlordai.game.player import LearningPlayer

Using TensorFlow backend.


In [2]:
class PreppedDataGenerator(Sequence):
    def __init__(self, path_id, batch_size=1024, timesteps_length=LearningPlayer.TIMESTEPS):
        self.path_id = path_id
        self.batch_size = batch_size
        self.timesteps_length = timesteps_length
        
        self.load_cache()
        self.on_epoch_end()
        
    def __len__(self):
        return len(self.cache[0]) // self.batch_size
    
    def on_epoch_end(self):
        p = np.random.permutation(len(self.cache[0]))
        
        all_history_matrices = self.cache[0][p]
        all_move_vectors = self.cache[1][p]
        all_hand_vectors = self.cache[2][p]
        all_y = self.cache[3][p]
        
        self.cache = (all_history_matrices, all_move_vectors, all_hand_vectors, all_y)
        
    def load_cache(self):
        with open(self.path_id, 'rb') as f:
            history_matrices, move_vectors, hand_vectors, y = pickle.load(f)
            
        self.cache = (history_matrices, move_vectors, hand_vectors, np.array(y)) 
        
        
    def __getitem__(self, index):
        """Generate one batch of data
        :param index: index of the batch
        :return: X and y when fitting. X only when predicting
        """
        
        history_matrices = self.cache[0][index * self.batch_size: (index + 1) * self.batch_size]
        move_vectors = self.cache[1][index * self.batch_size: (index + 1) * self.batch_size]
        hand_vectors = self.cache[2][index * self.batch_size: (index + 1) * self.batch_size]
        y = self.cache[3][index * self.batch_size: (index + 1) * self.batch_size]
        
        #return [self.densify(history_matrices), move_vectors, hand_vectors], y
        return [history_matrices, move_vectors, hand_vectors], y

In [3]:
data_folder = '4_10_actualq1'

In [4]:
assert data_folder is not None
!rm -r ../data/{data_folder}_agg
!gsutil -m cp -r gs://landlord_ai/{data_folder}_agg/ ../data/

Copying gs://landlord_ai/4_10_actualq1_agg/test.pkl...
Copying gs://landlord_ai/4_10_actualq1_agg/train.pkl...
\ [2/2 files][ 15.5 GiB/ 15.5 GiB] 100% Done 125.6 MiB/s ETA 00:00:00           
Operation completed over 2 objects/15.5 GiB.                                     


In [5]:
train_gen = PreppedDataGenerator('../data/' + data_folder + '_agg/train.pkl', batch_size=1 << 11)
test_gen = PreppedDataGenerator('../data/' + data_folder + '_agg/test.pkl', batch_size=1 << 11)

In [6]:
assert not np.allclose(train_gen[1][0][0], train_gen[0][0][0])
assert len(train_gen[0][0]) == 3
for i in range(3):
    get_set = train_gen[0][0][0]
    if len(get_set.shape) != 3:
        print(get_set)


In [7]:
def create_model_bidi():
    K.clear_session()
    GRU_DIM = 128

    history_inp = Input((None, LearningPlayer.TIMESTEP_FEATURES), name='history_inp')
    move_inp = Input((LearningPlayer.TIMESTEP_FEATURES, ), name='move_inp')
    hand_inp = Input((LearningPlayer.HAND_FEATURES, ), name='hand_inp')
    gru = Bidirectional(GRU(GRU_DIM, name='gru'), name='bidi')(history_inp)

    concat = Concatenate()([gru, move_inp, hand_inp])
    hidden1 = Dense(256, activation='relu', name='hidden1')(concat)
    hidden2 = Dense(128, activation='relu', name='hidden2')(BatchNormalization(name='bn1')(hidden1))
    hidden3 = Dense(64, activation='relu', name='hidden3')(BatchNormalization(name='bn2')(hidden2))

    output = Dense(1, activation='linear', name='output')(BatchNormalization(name='bn3')(hidden3))
    combined_net = keras.models.Model(inputs=[history_inp, move_inp, hand_inp], outputs=output)
    combined_net.compile(loss=keras.losses.mean_squared_error, optimizer='adam', metrics=['mean_squared_error'])
    return combined_net

In [8]:
import subprocess
def has_layer(model, layer):
    try:
        model.get_layer(layer)
        return True
    except:
        return False

def split_model_triage(composite, model_folder):
    best_model = keras.models.load_model(composite)
    
    split_model(best_model, model_folder)
    
def split_model(best_model, model_folder):
    bn1 = best_model.get_layer('bn1')
    bn2 = best_model.get_layer('bn2')
    bn3 = best_model.get_layer('bn3')
    history_net = keras.models.Model(inputs=[best_model.get_layer('history_inp').input], outputs=[best_model.get_layer('bidi').output])

    vector_history_inp = Input((best_model.get_layer('bidi').output.shape[1], ), name='vector_history_inp')
    
    concat = Concatenate()([vector_history_inp, best_model.get_layer('move_inp').output, best_model.get_layer('hand_inp').output])
    hidden1 = best_model.get_layer('hidden1')(concat)
    hidden2 = best_model.get_layer('hidden2')(bn1(hidden1))
    hidden3 = best_model.get_layer('hidden3')(bn2(hidden2))
    output = best_model.get_layer('output')(bn3(hidden3))

    move_inp = best_model.get_layer('move_inp').input
    hand_inp = best_model.get_layer('hand_inp').input
    position_net = keras.models.Model(inputs=[vector_history_inp, move_inp, hand_inp], outputs=[output])

    history_net.save(str(model_folder / 'history.h5'))
    position_net.save(str(model_folder / 'position.h5'))

In [9]:
def sanity_check_model(combined_file, net_dir):
    sanity_set = train_gen[0]
    historical_features, move_vectors, hand_vectors = sanity_set[0]
    targets = sanity_set[1]

    player = LearningPlayer(name='sanity', net_dir=str(net_dir))
    
    historical_matrix = player.history_net.predict(historical_features, batch_size=1024)

    from sklearn import metrics
    
    error_1 = metrics.mean_squared_error(targets, player.get_position_predictions(historical_matrix, move_vectors, hand_vectors))
    
    composite = keras.models.load_model(combined_file)
    error_2 = metrics.mean_squared_error(targets, composite.predict([historical_features, move_vectors, hand_vectors], batch_size=1024))
    print(combined_file, error_1, error_2)
    assert np.abs(error_1 - error_2) < 1E-2

In [10]:
from pathlib import Path

def delete_dir(path):
    if not os.path.exists(path):
        return
    for file in path.iterdir():
        os.remove(file)
    path.rmdir()
    
def publish_model(i):
    combined_file = data_folder + '_combined_' + str(i) + '.h5'
    if os.path.exists(combined_file):
        model_folder_name = data_folder + '_model' + str(i)

        model_folder_path = Path('../models/', model_folder_name)
        delete_dir(model_folder_path)
        model_folder_path.mkdir()

        split_model_triage(combined_file, model_folder_path)
        sanity_check_model(combined_file, model_folder_path)
        print(model_folder_name)
        subprocess.check_output(['gsutil', 'cp', '-r', '../models/' + model_folder_name + '/*', 'gs://landlord_ai/models/' + model_folder_name])

In [11]:
def train_model(fname='model.h5'):
    combined_net = create_model_bidi()
    
    callbacks = [
        EarlyStopping(monitor='val_mean_squared_error', mode='min', verbose=1, patience=3),
        ModelCheckpoint(fname, monitor='val_mean_squared_error', mode='min', verbose=1, save_best_only=True)
    ]

    combined_net.fit_generator(train_gen,
                                steps_per_epoch=3000,
                epochs=50,
                callbacks=callbacks,
                validation_data=test_gen,
                shuffle=True,
                workers=1,
                max_queue_size=10,
                use_multiprocessing=False
              )
    return combined_net

for i in range(14, 20):
    train_model(data_folder + '_combined_' + str(i) + '.h5')
    publish_model(i)

Epoch 1/50
3000/3000 [==============================] - 491s 164ms/step - loss: 0.9706 - mean_squared_error: 0.9706 - val_loss: 0.5127 - val_mean_squared_error: 0.4902

Epoch 00001: val_mean_squared_error improved from inf to 0.49016, saving model to 4_10_actualq1_combined_14.h5
Epoch 2/50
3000/3000 [==============================] - 495s 165ms/step - loss: 0.3283 - mean_squared_error: 0.3283 - val_loss: 0.2712 - val_mean_squared_error: 0.3634

Epoch 00002: val_mean_squared_error improved from 0.49016 to 0.36341, saving model to 4_10_actualq1_combined_14.h5
Epoch 3/50
3000/3000 [==============================] - 487s 162ms/step - loss: 0.2667 - mean_squared_error: 0.2667 - val_loss: 0.2459 - val_mean_squared_error: 0.3346

Epoch 00003: val_mean_squared_error improved from 0.36341 to 0.33459, saving model to 4_10_actualq1_combined_14.h5
Epoch 4/50
3000/3000 [==============================] - 495s 165ms/step - loss: 0.2419 - mean_squared_error: 0.2419 - val_loss: 0.2053 - val_mean_square

/opt/conda/lib/python3.7/site-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


4_10_actualq1_combined_14.h5 0.18332753 0.18332753
4_10_actualq1_model14
Epoch 1/50
3000/3000 [==============================] - 488s 163ms/step - loss: 0.8727 - mean_squared_error: 0.8727 - val_loss: 0.3879 - val_mean_squared_error: 0.3279

Epoch 00001: val_mean_squared_error improved from inf to 0.32790, saving model to 4_10_actualq1_combined_15.h5
Epoch 2/50
3000/3000 [==============================] - 493s 164ms/step - loss: 0.3069 - mean_squared_error: 0.3069 - val_loss: 0.3525 - val_mean_squared_error: 0.3709

Epoch 00002: val_mean_squared_error did not improve from 0.32790
Epoch 3/50
3000/3000 [==============================] - 488s 163ms/step - loss: 0.2599 - mean_squared_error: 0.2599 - val_loss: 0.3355 - val_mean_squared_error: 0.3100

Epoch 00003: val_mean_squared_error improved from 0.32790 to 0.30996, saving model to 4_10_actualq1_combined_15.h5
Epoch 4/50
3000/3000 [==============================] - 496s 165ms/step - loss: 0.2385 - mean_squared_error: 0.2385 - val_loss: 0

/opt/conda/lib/python3.7/site-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


4_10_actualq1_combined_15.h5 0.20493196 0.20493197
4_10_actualq1_model15
Epoch 1/50
3000/3000 [==============================] - 490s 163ms/step - loss: 0.9123 - mean_squared_error: 0.9123 - val_loss: 0.3892 - val_mean_squared_error: 0.4696

Epoch 00001: val_mean_squared_error improved from inf to 0.46965, saving model to 4_10_actualq1_combined_16.h5
Epoch 2/50
3000/3000 [==============================] - 495s 165ms/step - loss: 0.3019 - mean_squared_error: 0.3019 - val_loss: 0.3791 - val_mean_squared_error: 0.3882

Epoch 00002: val_mean_squared_error improved from 0.46965 to 0.38821, saving model to 4_10_actualq1_combined_16.h5
Epoch 3/50
3000/3000 [==============================] - 489s 163ms/step - loss: 0.3244 - mean_squared_error: 0.3244 - val_loss: 0.2833 - val_mean_squared_error: 0.2909

Epoch 00003: val_mean_squared_error improved from 0.38821 to 0.29089, saving model to 4_10_actualq1_combined_16.h5
Epoch 4/50
3000/3000 [==============================] - 497s 166ms/step - loss:

...

4_10_actualq1_combined_17.h5 0.24334693 0.24334693
4_10_actualq1_model17
Epoch 1/50
3000/3000 [==============================] - 490s 163ms/step - loss: 0.9202 - mean_squared_error: 0.9202 - val_loss: 0.4724 - val_mean_squared_error: 0.4121

Epoch 00001: val_mean_squared_error improved from inf to 0.41212, saving model to 4_10_actualq1_combined_18.h5
Epoch 2/50
3000/3000 [==============================] - 494s 165ms/step - loss: 0.3064 - mean_squared_error: 0.3064 - val_loss: 0.3063 - val_mean_squared_error: 0.2753

Epoch 00002: val_mean_squared_error improved from 0.41212 to 0.27534, saving model to 4_10_actualq1_combined_18.h5
Epoch 3/50
3000/3000 [==============================] - 485s 162ms/step - loss: 0.2624 - mean_squared_error: 0.2624 - val_loss: 0.4779 - val_mean_squared_error: 0.3888

Epoch 00003: val_mean_squared_error did not improve from 0.27534
Epoch 4/50
3000/3000 [==============================] - 494s 165ms/step - loss: 0.2368 - mean_squared_error: 0.2368 - val_loss: 0

/opt/conda/lib/python3.7/site-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


4_10_actualq1_combined_18.h5 0.2515535 0.25155348
4_10_actualq1_model18
Epoch 1/50
3000/3000 [==============================] - 489s 163ms/step - loss: 0.8057 - mean_squared_error: 0.8057 - val_loss: 0.2819 - val_mean_squared_error: 0.3803

Epoch 00001: val_mean_squared_error improved from inf to 0.38027, saving model to 4_10_actualq1_combined_19.h5
Epoch 2/50
3000/3000 [==============================] - 492s 164ms/step - loss: 0.3062 - mean_squared_error: 0.3062 - val_loss: 0.3541 - val_mean_squared_error: 0.3440

Epoch 00002: val_mean_squared_error improved from 0.38027 to 0.34401, saving model to 4_10_actualq1_combined_19.h5
Epoch 3/50
3000/3000 [==============================] - 487s 162ms/step - loss: 0.2604 - mean_squared_error: 0.2604 - val_loss: 0.2161 - val_mean_squared_error: 0.2754

Epoch 00003: val_mean_squared_error improved from 0.34401 to 0.27544, saving model to 4_10_actualq1_combined_19.h5
Epoch 4/50
3000/3000 [==============================] - 495s 165ms/step - loss: 

/opt/conda/lib/python3.7/site-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


4_10_actualq1_combined_19.h5 0.11679556 0.116795585
4_10_actualq1_model19


In [34]:
publish_model(10)

/opt/conda/lib/python3.7/site-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


4_7_actualq3_combined_10.h5 0.17735313047530865 0.1773531122255772
4_7_actualq3_model10
